In [41]:
from pyspark import SparkContext
import time

In [21]:
from os import listdir
import re

In [12]:
directory = './data/indexing/'

In [13]:
files = [f for f in listdir(directory)]

In [20]:
files

['32',
 '35',
 '34',
 '33',
 '20',
 '18',
 '27',
 '9',
 '0',
 '11',
 '7',
 '29',
 '16',
 '42',
 '6',
 '28',
 '17',
 '1',
 '10',
 '19',
 '26',
 '8',
 '21',
 '44',
 '43',
 '38',
 '36',
 '31',
 '30',
 '37',
 '39',
 '41',
 '24',
 '23',
 '4',
 '15',
 '3',
 '12',
 '40',
 '2',
 '13',
 '5',
 '14',
 '22',
 '25']

In [25]:
class Word_dict:
    def __init__(self):
        self.last = 0
        self.dict = {}
    def add(self, key):
        self.dict[key] = self.last
        self.last += 1

In [28]:
words = Word_dict()

In [31]:
def add_to_dictionary(line, word_dict):
    for word in re.findall(r'\w+', line):
        if word not in word_dict.dict.keys():
            word_dict.add(word)

In [38]:
words = Word_dict()
for file_name in files:
    with open(directory+file_name, encoding = "ISO-8859-1") as f:
        for line in f:
            # print(line)
            add_to_dictionary(line, words)

In [39]:
len(words.dict.keys())

115128

In [134]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import CountVectorizer
import re
from pyspark.sql.functions import input_file_name

In [51]:
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-43-2dfc28fca47d>:1 

In [52]:
sql = SparkSession(sc)

In [58]:
df = sql.createDataFrame([(0, ["a", "b", "c", "d"]), (1, ["a", "e", "b", "c", "a"])],["label", "raw"])
cv = CountVectorizer(inputCol="raw", outputCol="vectors")
model = cv.fit(df)
model.transform(df).show(truncate=False)

+-----+---------------+-------------------------------+
|label|raw            |vectors                        |
+-----+---------------+-------------------------------+
|0    |[a, b, c, d]   |(5,[0,1,2,4],[1.0,1.0,1.0,1.0])|
|1    |[a, e, b, c, a]|(5,[0,1,2,3],[2.0,1.0,1.0,1.0])|
+-----+---------------+-------------------------------+



In [93]:
model.transform(df).select('vectors').show()

+--------------------+
|             vectors|
+--------------------+
|(5,[0,1,2,4],[1.0...|
|(5,[0,1,2,3],[2.0...|
+--------------------+



In [105]:
folder = './data/mini_reverse'
a = 0

In [110]:
def dictionary_builder(word):
    global a
    a += 1
    return (word,a-1)

In [119]:
text = sc.textFile(folder)
text

./data/mini_reverse MapPartitionsRDD[196] at textFile at NativeMethodAccessorImpl.java:0

In [115]:
text.flatMap(lambda x: re.findall(r'\w+', x)).map(lambda x: (x,1)).reduceByKey(lambda x,y: max(x,y)).collect()
#reduceByKey(lambda x,y: max(x,y)).collect()

[('there', 1),
 ('General', 1),
 ('Kenobi', 1),
 ('needs', 1),
 ('of', 1),
 ('came', 1),
 ('say', 1),
 ('the', 1),
 ('general', 1),
 ('to', 1),
 ('hello', 1),
 ('Hello', 1),
 ('World', 1),
 ('am', 1),
 ('new', 1),
 ('The', 1),
 ('world', 1),
 ('more', 1),
 ('I', 1),
 ('disagree', 1)]

In [129]:
text.flatMap(lambda x: re.findall(r'\w+', x)).distinct().collect()#.withColumn("uniqueID",monotonicallyIncreasingId)

['there',
 'General',
 'Kenobi',
 'needs',
 'of',
 'came',
 'say',
 'the',
 'general',
 'to',
 'hello',
 'Hello',
 'World',
 'am',
 'new',
 'The',
 'world',
 'more',
 'I',
 'disagree']

In [116]:
df2 = sql.createDataFrame([(0, text.flatMap(lambda x: re.findall(r'\w+', x)).collect())], ['label','raw'])
cv = CountVectorizer(inputCol="raw", outputCol="vectors")
model2 = cv.fit(df2)
test_vector = model2.transform(df2)
test_vector.show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------+
|label|raw                                                                                                                                                          |vectors                                                                                                                                   |
+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[Hello, there, General, Kenobi, Hello, World, I, am, the, new, general, The, w

In [113]:
output = text.flatMap(lambda file,contents:[(file, word) for word in contents.lower().split()])\
            .map(lambda word: (word, 1))\
            .map(lambda file, word: (word,[file]))\
            .reduceByKey(lambda a,b: a+b)

In [131]:
type(text)

pyspark.rdd.RDD

In [140]:
whole_text = sc.wholeTextFiles(folder)
whole_text.collect()

[('file:/Users/luzjubierrezapater/insight_projects/interview_prep/coding_challenges/coding_challenges/data/mini_reverse/0',
  'Hello there, General Kenobi'),
 ('file:/Users/luzjubierrezapater/insight_projects/interview_prep/coding_challenges/coding_challenges/data/mini_reverse/1',
  'Hello World, I am the new general'),
 ('file:/Users/luzjubierrezapater/insight_projects/interview_prep/coding_challenges/coding_challenges/data/mini_reverse/3',
  'The world needs more of Kenobi.\nI disagree'),
 ('file:/Users/luzjubierrezapater/insight_projects/interview_prep/coding_challenges/coding_challenges/data/mini_reverse/2',
  'The general came to say hello')]

In [195]:
whole_text.map(lambda x: (x[0].split('/')[-1], re.findall(r'\w+', x[1])))\
.flatMapValues(lambda x: x)\
.map(lambda x: (x[1],list(x[0])))\
.reduceByKey(lambda x,y: x+y)\
.saveAsTextFile('output.txt')
#.collect()

#.flatMap(lambda file,contents: [(file, word) for word in contents]).collect()

#.map(lambda x: (re.findall(r'\w+', x[1]),x[0].split('/')[-1])).flatMap(lambda x: ).collect()



#.map(lambda x: (re.findall(r'\w+', x[0]),x[1])).select(explode())

#.collect()

AttributeError: 'list' object has no attribute 'saveAsTextFile'